In [4]:
import pandas as pd
from datetime import date, timedelta
import os
import requests

In [5]:
url_token = "https://identity.dataspace.copernicus.eu/auth/realms/CDSE/protocol/openid-connect/token"

payload = {
    'grant_type': 'client_credentials',
    'client_id': 'sh-f4a12c5e-c495-4abc-ba13-41ed2bc9f78d',
    'client_secret': 'sKkV8jwwlum5m1LcL4kdmGGyvlIqvAZV'
}

response = requests.post(url_token, data=payload)

if response.status_code == 200:
    token = response.json()['access_token']
    print("✅ Token obtenido:", token)
else:
    print("❌ Error al obtener token:", response.text)


✅ Token obtenido: eyJhbGciOiJSUzI1NiIsInR5cCIgOiAiSldUIiwia2lkIiA6ICJYVUh3VWZKaHVDVWo0X3k4ZF8xM0hxWXBYMFdwdDd2anhob2FPLUxzREZFIn0.eyJleHAiOjE3NTEyMzIyMDUsImlhdCI6MTc1MTIzMTYwNSwianRpIjoiMTJlMGQ5NzUtYjdlMC00MDhhLTljOTAtNDdhOThjN2Y2M2IxIiwiaXNzIjoiaHR0cHM6Ly9pZGVudGl0eS5kYXRhc3BhY2UuY29wZXJuaWN1cy5ldS9hdXRoL3JlYWxtcy9DRFNFIiwic3ViIjoiZmU5OTVlMjgtNDk4YS00NWJhLWEyN2UtNzE4NjhiMDFhZjgwIiwidHlwIjoiQmVhcmVyIiwiYXpwIjoic2gtZjRhMTJjNWUtYzQ5NS00YWJjLWJhMTMtNDFlZDJiYzlmNzhkIiwic2NvcGUiOiJlbWFpbCBwcm9maWxlIHVzZXItY29udGV4dCIsImVtYWlsX3ZlcmlmaWVkIjpmYWxzZSwiY2xpZW50SG9zdCI6IjE5MS4xNTYuMzMuMTY1Iiwib3JnYW5pemF0aW9ucyI6WyJkZWZhdWx0LWQwZTRkYzc1LWQ0YWUtNDBjMS1hNjdlLTYwMzkwNzk2MmQyYiJdLCJ1c2VyX2NvbnRleHRfaWQiOiJiMjc3ZDVlMS1kM2M4LTRhNzItOTc2Yy1hNTUwOGVkMmNmOWIiLCJjb250ZXh0X3JvbGVzIjp7fSwiY29udGV4dF9ncm91cHMiOlsiL2FjY2Vzc19ncm91cHMvdXNlcl90eXBvbG9neS9jb3Blcm5pY3VzX2dlbmVyYWwvIiwiL29yZ2FuaXphdGlvbnMvZGVmYXVsdC1kMGU0ZGM3NS1kNGFlLTQwYzEtYTY3ZS02MDM5MDc5NjJkMmIvIl0sInByZWZlcnJlZF91c2VybmFtZSI6InNlcnZpY2UtYWNjb3

In [6]:
# # Hacer solicitud
# headers = {"Authorization": f"Bearer {token}"}

# response = requests.get(url_search, headers=headers)
# print(f"Status Code: {response.status_code}")
# print(f"Response Text: {response.text}")
# if response.status_code == 200:
#     products = response.json()['value']
#     print(f"✅ Encontrados {len(products)} productos")
# else:
#     print("❌ Error al consultar el catálogo:", response.text)
#     exit()

In [7]:
import requests
import logging
from datetime import datetime, timedelta

# Definir la URL con la nueva fecha
nueva_fecha_inicio = "2024-01-01"  # Cambia esta fecha según tus necesidades
nueva_fecha_fin = "2024-12-31"     # Cambia esta fecha según tus necesidades
fecha_inicio = datetime.strptime(nueva_fecha_inicio, '%Y-%m-%d') #- timedelta(days=1)).strftime('%Y-%m-%d')
fecha_fin = datetime.strptime(nueva_fecha_fin, '%Y-%m-%d') #+ timedelta(days=1)).strftime('%Y-%m-%d')
aoi = "POLYGON((-0.5 39.5, 0.5 39.5, 0.5 40.5, -0.5 40.5, -0.5 39.5))"
url_search = f"https://catalogue.dataspace.copernicus.eu/odata/v1/Products?$filter=Collection/Name eq 'SENTINEL-1' and OData.CSC.Intersects(area=geography'SRID=4326;{aoi}') and ContentDate/Start gt {fecha_inicio}T00:00:00.000Z and ContentDate/Start lt {fecha_fin}T00:00:00.000Z&$top=1000"

In [8]:
import requests
import urllib.parse
import logging

logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# token = "TU_TOKEN"
base_url = "https://catalogue.dataspace.copernicus.eu/odata/v1/Products"

# Define el polígono en formato WKT
polygon_wkt = "SRID=4326;POLYGON((-0.5 39.5, 0.5 39.5, 0.5 40.5, -0.5 40.5, -0.5 39.5))"

# Codifica correctamente la geometría y el filtro completo
intersects_filter = f"OData.CSC.Intersects(area=geography'{polygon_wkt}')"
full_filter = f"Collection/Name eq 'SENTINEL-1' and {intersects_filter} and ContentDate/Start gt 2024-01-01T00:00:00.000Z and ContentDate/Start lt 2024-12-31T00:00:00.000Z"

# Codificar la URL
params = {
    "$filter": full_filter,
    "$top": 1000
}
encoded_params = urllib.parse.urlencode(params, safe="():,' ")

url_search = f"{base_url}?{encoded_params}"

headers = {"Authorization": f"Bearer {token}"}

try:
    response = requests.get(url_search, headers=headers)
    response.raise_for_status()
    logging.info(f"Status Code: {response.status_code}")

    if response.headers.get('Content-Type') and 'application/json' in response.headers.get('Content-Type'):
        data = response.json()
        if 'value' in data:
            products = data['value']
            logging.info(f"✅ Encontrados {len(products)} productos")
            for i, product in enumerate(products[:5]):
                logging.info(f"Producto {i+1}: {product}")
        else:
            logging.error("❌ La respuesta JSON no contiene la clave 'value'")
    else:
        logging.error("❌ La respuesta no es JSON")

except requests.exceptions.HTTPError as http_err:
    logging.error(f"❌ Error HTTP: {http_err}")
    if response.status_code == 401:
        logging.error("Token de acceso inválido o expirado. Intenta refrescar el token.")
    elif response.status_code == 404:
        logging.error("Recurso no encontrado. Verifica la URL.")
    elif response.status_code == 500:
        logging.error("Error en el servidor. Intenta más tarde.")
    else:
        logging.error(f"Error {response.status_code}: {response.text}")
except requests.exceptions.RequestException as req_err:
    logging.error(f"❌ Error en la solicitud: {req_err}")
except ValueError as json_err:
    logging.error(f"❌ Error al parsear JSON: {json_err}")
except Exception as e:
    logging.error(f"❌ Error inesperado: {e}")


2025-06-29 16:13:40,587 - INFO - Status Code: 200
2025-06-29 16:13:40,599 - INFO - ✅ Encontrados 920 productos
2025-06-29 16:13:40,600 - INFO - Producto 1: {'@odata.mediaContentType': 'application/octet-stream', 'Id': 'fac97b1c-c4ec-4273-b39e-97954ef73bd5', 'Name': 'S1A_IW_GRDH_1SDV_20240210T180320_20240210T180345_052500_065989_597E.SAFE', 'ContentType': 'application/octet-stream', 'ContentLength': 1747447531, 'OriginDate': '2024-02-10T18:47:24.290000Z', 'PublicationDate': '2024-02-10T18:53:51.067273Z', 'ModificationDate': '2024-02-10T18:54:04.748909Z', 'Online': True, 'EvictionDate': '9999-12-31T23:59:59.999999Z', 'S3Path': '/eodata/Sentinel-1/SAR/IW_GRDH_1S/2024/02/10/S1A_IW_GRDH_1SDV_20240210T180320_20240210T180345_052500_065989_597E.SAFE', 'Checksum': [{'Value': 'c45b676c7680659a652834cd0b523e85', 'Algorithm': 'MD5', 'ChecksumDate': '2024-02-10T18:54:01.714173Z'}, {'Value': '755f8db999014f89a56853ceabe1432b90536f6917498720a49020a09c1fde64', 'Algorithm': 'BLAKE3', 'ChecksumDate': '2

In [9]:
# Convertir a DataFrame
if products:
    df = pd.DataFrame.from_dict(products)
    print("Productos disponibles:")
    print(df[['Name', 'Id', 'ContentDate']])
else:
    print("⚠️ No se encontraron productos")

Productos disponibles:
                                                  Name  \
0    S1A_IW_GRDH_1SDV_20240210T180320_20240210T1803...   
1    S1A_IW_GRDH_1SDV_20240210T180255_20240210T1803...   
2    S1A_IW_GRDH_1SDV_20240228T061016_20240228T0610...   
3    S1A_IW_RAW__0SDV_20240210T180316_20240210T1803...   
4    S1A_IW_RAW__0SDV_20240210T180251_20240210T1803...   
..                                                 ...   
915  S1A_IW_SLC__1SDV_20241224T061011_20241224T0610...   
916  S1A_IW_ETA__AXDV_20241219T060202_20241219T0602...   
917  S1A_IW_ETA__AXDV_20241218T180316_20241218T1803...   
918  S1A_IW_ETA__AXDV_20241218T180251_20241218T1803...   
919  S1A_IW_GRDH_1SDV_20241031T180256_20241031T1803...   

                                       Id  \
0    fac97b1c-c4ec-4273-b39e-97954ef73bd5   
1    81c8dff9-05d1-4b0c-a5ac-4b39f7708503   
2    38d6494d-3ad6-4fd4-b3d4-5e2c54326036   
3    e6fe18a7-dfa5-4fab-867e-dbdb60498d30   
4    d3028802-79af-4836-982f-18dbf56d26eb   
..        

In [16]:
import pandas as pd

# Simulamos tu DataFrame con una sola columna de diccionarios
df = pd.DataFrame({'product': 
                   list(products[0])  # Aquí usamos la lista de productos obtenida
                   })


In [19]:
products[0]['GeoFootprint']

{'type': 'Polygon',
 'coordinates': [[[-2.924355, 39.567089],
   [-0.0, 39.96116848410655],
   [0.073328, 39.97105],
   [-0.0, 40.3259385103068],
   [-0.236807, 41.472023],
   [-3.302319, 41.06934],
   [-2.924355, 39.567089]]]}

In [22]:
len(products)

920

In [23]:
import folium
from shapely import wkt
from shapely.geometry import mapping
import json

# Crea un mapa centrado aproximadamente en tu polígono
m = folium.Map(location=[40.0, 0.0], zoom_start=8)

# Itera sobre los productos
for product in products:
    try:
        # Supongamos que el footprint viene en formato WKT
        footprint_wkt = product.get('Footprint') or product.get('Geometry') or product.get('geoFootprint')
        if not footprint_wkt:
            continue
        
        geom = wkt.loads(footprint_wkt)
        geo_json = mapping(geom)

        folium.GeoJson(
            geo_json,
            tooltip=product.get('Name', 'Producto Sentinel-1'),
            style_function=lambda x: {'color': 'blue', 'weight': 2, 'fillOpacity': 0.1}
        ).add_to(m)
    except Exception as e:
        print(f"Error procesando producto: {e}")

# Guarda como archivo HTML
m.save("sentinel1_map.html")
print("✅ Mapa guardado como 'sentinel1_map.html'")


Error procesando producto: ParseException: Unknown type: 'GEOGRAPHY'SRID=4326;POLYGON'
Error procesando producto: ParseException: Unknown type: 'GEOGRAPHY'SRID=4326;POLYGON'
Error procesando producto: ParseException: Unknown type: 'GEOGRAPHY'SRID=4326;POLYGON'
Error procesando producto: ParseException: Unknown type: 'GEOGRAPHY'SRID=4326;POLYGON'
Error procesando producto: ParseException: Unknown type: 'GEOGRAPHY'SRID=4326;POLYGON'
Error procesando producto: ParseException: Unknown type: 'GEOGRAPHY'SRID=4326;POLYGON'
Error procesando producto: ParseException: Unknown type: 'GEOGRAPHY'SRID=4326;POLYGON'
Error procesando producto: ParseException: Unknown type: 'GEOGRAPHY'SRID=4326;POLYGON'
Error procesando producto: ParseException: Unknown type: 'GEOGRAPHY'SRID=4326;POLYGON'
Error procesando producto: ParseException: Unknown type: 'GEOGRAPHY'SRID=4326;POLYGON'
Error procesando producto: ParseException: Unknown type: 'GEOGRAPHY'SRID=4326;POLYGON'
Error procesando producto: ParseException: 

In [24]:
import webbrowser
webbrowser.open("sentinel1_map.html")


True

In [27]:
import folium
from shapely import wkt
from shapely.geometry import mapping

# Crea un mapa centrado en la región del polígono
m = folium.Map(location=[40.0, 0.0], zoom_start=8)

# Añade el polígono que usaste en la consulta (región de interés)
# Coordenadas del polígono (latitud, longitud)
polygon_coords = [
    [39.5, -0.5],
    [39.5, 0.5],
    [40.5, 0.5],
    [40.5, -0.5],
    [39.5, -0.5]
]

folium.Polygon(
    locations=polygon_coords,
    color='red',
    weight=2,
    fill=True,
    fill_color='red',
    fill_opacity=0.2,
    tooltip="Región consultada"
).add_to(m)

# Añade geometría de productos (si existe)
for product in products:
    try:
        # Usa la clave que contenga la geometría en WKT (ajusta si es diferente)
        footprint_wkt = product.get('Footprint') or product.get('Geometry') or product.get('geoFootprint')
        if not footprint_wkt:
            continue

        geom = wkt.loads(footprint_wkt)
        geo_json = mapping(geom)

        folium.GeoJson(
            geo_json,
            tooltip=product.get('Name', 'Producto Sentinel-1'),
            style_function=lambda x: {
                'color': 'blue',
                'weight': 2,
                'fillOpacity': 0.1
            }
        ).add_to(m)
    except Exception as e:
        print(f"Error procesando producto: {e}")



Error procesando producto: ParseException: Unknown type: 'GEOGRAPHY'SRID=4326;POLYGON'
Error procesando producto: ParseException: Unknown type: 'GEOGRAPHY'SRID=4326;POLYGON'
Error procesando producto: ParseException: Unknown type: 'GEOGRAPHY'SRID=4326;POLYGON'
Error procesando producto: ParseException: Unknown type: 'GEOGRAPHY'SRID=4326;POLYGON'
Error procesando producto: ParseException: Unknown type: 'GEOGRAPHY'SRID=4326;POLYGON'
Error procesando producto: ParseException: Unknown type: 'GEOGRAPHY'SRID=4326;POLYGON'
Error procesando producto: ParseException: Unknown type: 'GEOGRAPHY'SRID=4326;POLYGON'
Error procesando producto: ParseException: Unknown type: 'GEOGRAPHY'SRID=4326;POLYGON'
Error procesando producto: ParseException: Unknown type: 'GEOGRAPHY'SRID=4326;POLYGON'
Error procesando producto: ParseException: Unknown type: 'GEOGRAPHY'SRID=4326;POLYGON'
Error procesando producto: ParseException: Unknown type: 'GEOGRAPHY'SRID=4326;POLYGON'
Error procesando producto: ParseException: 

In [28]:
# Centrar el mapa en el centro del polígono
center_lat = sum([coord[0] for coord in polygon_coords]) / len(polygon_coords)
center_lon = sum([coord[1] for coord in polygon_coords]) / len(polygon_coords)
m.location = [center_lat, center_lon]


In [29]:

# Guarda el mapa como HTML
m.save("sentinel2_region_map.html")
print("✅ Mapa guardado como 'sentinel1_region_map.html'")

✅ Mapa guardado como 'sentinel1_region_map.html'
